<div align=center>
		
<p></p>
<p></p>
<font size=5>
In the Name of God
<font/>
<p></p>
 <br/>
    <br/>
    <br/>
<font color=#FF7500>
Sharif University of Technology - Departmenet of Computer Engineering
</font>
<p></p>
<font color=blue>
Artifical Intelligence - Dr. Mohammad Hossein Rohban
</font>
<br/>
<br/>
Fall 2021

</div>

<hr/>
		<div align=center>
		    <font color=red size=6>
			    <br />
Practical Assignment 5-Q1
            	<br/>
			</font>
    <br/>
    <br/>
<font size=4>
			<br/><br/>
Deadline:  Bahman 17th
                <br/><b>
              Cheating is Strongly Prohibited
                </b><br/><br/>
                <font color=red>
Please run all the cells.
     </font>
</font>
                <br/>
    </div>

# Personal Data

In [1]:
# Set your student number
student_number = 97101489
Name = 'Amirhosein'
Last_Name = 'Javadi'

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Rules
- You are not allowed to use provided codes that can be found on the internet. 
- If you want to use a library which is not already imported, you must ask a question on Quera to get the permission of using that.
- Do not hesitate to ask questions on Quera, if you have any.
- This assignment is due Bahman 17th 23:59:59. you can use up to 1 grace day for this assignment and the hard deadline is Bahman 18th 23:59:59.

# Q1 (30Points)

## Mountain Car

The OpenAI Gym library includes a set of Python Reinforcement Learning environments. In this question, we examine the Mountain Car environment of this collection. Mountain Car is a Reinforcement Learning task that aims to learn the policy of climbing a steep hill and reaching the flag-marked goal. Also, the car engine is not powerful enough to climb straight up the hill on the right. Therefore, it must gain enough acceleration by climbing the hill on the left.


In this case, the state of the car is determined by an array containing its position and speed.

| Num | Observation | Min | Max|
| --- | --- | --- | ---|
| 0 | Position | -1.2 | 0.6 |
| 1 | Velocity | -0.07 | 0.07|

The intelligent agent is allowed to perform three movements of right push, no push and left push in each step. The agent's move will be given to the environment and the environment returns the next state along with the movement reward. For each step that the car does not reach the target, a cost of -1 is considered. Use Q-learning to find the optimal policy in each case. To do this, you must complete the functions below.

** Note that you will receive the full score of this question only if you can achieve a score of -115 with your implementation.

<font size=4>
Author: Atoosa Chegini
			<br/>
                <font color=red>
Please run all the cells.
     </font>
</font>
                <br/>
    </div>

In [2]:
import numpy as np
import random
import gym

In [3]:
#OLD

env = gym.make("MountainCar-v0")
# initialization (parameters such as learning rate, gamma, number of states, number of actions, and q_table)
env.reset()

###############################

n_actions = env.action_space.n
state_dim = env.observation_space.shape

##############################

lr = 0.6
gamma = 1
pos_low, pos_high = -1.2, 0.7
vel_low, vel_high =  -0.07, 0.07

##############################

position_step = 0.01     #   = (1.2+0.7)/190
#position_quantization_arr = np.linspace(-1.2, 0.7, num=190)
#position_len = len(position_quantization_arr)
#print(position_step)

##############################

velocity_step = 0.001 #(0.07+0.07)/1000
#velocity_quantization_arr = np.linspace(-0.07, 0.07, num=1000)
#velocity_len = len(velocity_quantization_arr)
#print(velocity_step)

##############################

num_states = 190*1000
q_table = np.zeros(shape=(num_states, n_actions))

##############################

print("position step:",position_step)
print("velocity_step:",velocity_step)
print("shape of q_table:",np.shape(q_table))

##############################

position step: 0.01
velocity_step: 0.001
shape of q_table: (190000, 3)


Next cell wants you supplement two functions. First for transforming the continuous space into discrete one (in order to make using q_table feasible), second for updating q_values based on the last action done by agent.

In [4]:
# This is just one example of a discretization function. You can change it as you want.
def discretize_state(x, minn, step):
    return int((x - minn) / step)

def env_state_to_Q_state(state, position_step = position_step, velocity_step = velocity_step):
    [position, velocity] = state
    pos_low, pos_high = -1.2, 0.7
    vel_low, vel_high =  -0.07, 0.07
    #print(position_step, velocity_step)
    position_pos = discretize_state(position, pos_low, position_step)
    velocity_pos = discretize_state(velocity, vel_low, velocity_step)
    return position_pos * 1000 + velocity_pos
    # Complete this function!

####################  test ####################

print(env_state_to_Q_state([-1.2,-0.07]))
print(env_state_to_Q_state([0.7 ,-0.07]))
print(env_state_to_Q_state([-1.2, 0.07]))
print(env_state_to_Q_state([0.7 , 0.07]))

####################  test ####################

def update_q(q_table, current_state, action, reward, next_state, lr=lr):
    best_future_q = np.max(q_table[next_state])

    ##############################################

    best_future_r = reward + gamma * best_future_q
    q_table[current_state][action] = (1 - lr) * q_table[current_state][action] + lr * best_future_r

    ##############################################

    # Complete this function!

0
190000
140
190140


At the following cell, the ends of two functions are getting current action based on the policy and defining the training process respectively.

In [5]:
# You may change the inputs of any function as you desire.
def get_action(q_table, current_state, epsilon):
    p = np.random.random()
    if p < epsilon:
        return np.random.randint(0, n_actions)
    else:
        return np.argmax(q_table[current_state])

def q_learning():
    lr = 0.6
    gamma = 1
    episodes = 100000
    reward_list = []
    epsilon = 1
    for episode_counter in range(episodes):
        state = env_state_to_Q_state(env.reset())
        total_reward = 0
        done = False
        step = 0
        while not done:
            action = get_action(q_table, state, epsilon=epsilon)
            new_state, reward, done, _ = env.step(action)
            new_state = env_state_to_Q_state(new_state)
            update_q(q_table, state, action, reward, new_state, lr)
            state = new_state
            total_reward += reward

        reward_list.append(total_reward)
        epsilon -= epsilon / episodes
        if (episode_counter + 1) % 5000 == 0:
            print("Episode:", episode_counter + 1, ',reward:', total_reward, ',epsilon:', epsilon)

#           with open('/content/drive/MyDrive/Colab Notebooks/Q1/Save_Files/q_table.npy', 'wb') as f:
#               np.save(f, q_table)

def save_policy():
    policy  = []
    for item in q_table:
        policy.append(np.argmax(item))
    np.save('policy.npy', policy)

In [6]:
q_learning()
save_policy()

Episode: 5000 ,reward: -200.0 ,epsilon: 0.9512291866918031
Episode: 10000 ,reward: -200.0 ,epsilon: 0.904836965614347
Episode: 15000 ,reward: -200.0 ,epsilon: 0.8607073308900115
Episode: 20000 ,reward: -200.0 ,epsilon: 0.8187299343421797
Episode: 25000 ,reward: -200.0 ,epsilon: 0.7787998095645411
Episode: 30000 ,reward: -200.0 ,epsilon: 0.7408171094478084
Episode: 35000 ,reward: -200.0 ,epsilon: 0.7046868565074071
Episode: 40000 ,reward: -200.0 ,epsilon: 0.6703187053879416
Episode: 45000 ,reward: -200.0 ,epsilon: 0.637626716950477
Episode: 50000 ,reward: -200.0 ,epsilon: 0.6065291433777642
Episode: 55000 ,reward: -200.0 ,epsilon: 0.5769482237601073
Episode: 60000 ,reward: -200.0 ,epsilon: 0.5488099896506059
Episode: 65000 ,reward: -200.0 ,epsilon: 0.5220440801036845
Episode: 70000 ,reward: -200.0 ,epsilon: 0.4965835657342961
Episode: 75000 ,reward: -200.0 ,epsilon: 0.47236478135794846
Episode: 80000 ,reward: -196.0 ,epsilon: 0.4493271667929729
Episode: 85000 ,reward: -200.0 ,epsilon: 0

In [7]:
# Attention: don't change this function. we will use this to grade your policy which you will hand in with policy.npy
# btw you can use it to see how you are performing. Uncomment two lines which are commented to be able to see what is happening visually.
def score():
    policy, scores = np.load("policy.npy"), []
    print(policy, scores)
    for episode in range(1000):
        print(f"******Episode {episode}")
        state, score, done, step = env_state_to_Q_state(env.reset(), position_step, velocity_step ), 0, False, 0
        print(state)
        while not done:
            # time.sleep(0.04)
            action = policy[state]
            state, reward, done, _ = env.step(action)
            state = env_state_to_Q_state(state, position_step, velocity_step)
            step += 1
            score += int(reward)
            # env.render()
        print(f"Score:{score}")
        scores.append(score)
    print(f"Average score over 1000 run : {np.array(scores).mean()}")

score()

[0 0 0 ... 0 0 0] []
******Episode 0
77070
Score:-92
******Episode 1
74070
Score:-151
******Episode 2
67070
Score:-116
******Episode 3
77070
Score:-90
******Episode 4
75070
Score:-168
******Episode 5
67070
Score:-113
******Episode 6
70070
Score:-152
******Episode 7
73070
Score:-151
******Episode 8
67070
Score:-112
******Episode 9
74070
Score:-154
******Episode 10
72070
Score:-148
******Episode 11
67070
Score:-110
******Episode 12
69070
Score:-106
******Episode 13
62070
Score:-116
******Episode 14
65070
Score:-115
******Episode 15
68070
Score:-112
******Episode 16
65070
Score:-117
******Episode 17
75070
Score:-167
******Episode 18
74070
Score:-151
******Episode 19
77070
Score:-95
******Episode 20
63070
Score:-117
******Episode 21
72070
Score:-148
******Episode 22
63070
Score:-118
******Episode 23
64070
Score:-115
******Episode 24
79070
Score:-96
******Episode 25
67070
Score:-116
******Episode 26
76070
Score:-93
******Episode 27
62070
Score:-119
******Episode 28
70070
Score:-152
******Ep